In [1]:
import pandas as pd
import numpy as np
import pickle as p

In [2]:
connectome = pd.read_csv("connectome_raw_data/Connectome.csv",sep=";")
muscles = pd.read_csv("connectome_raw_data/NeuronsToMuscle.csv",sep=";")
sensor = pd.read_csv("connectome_raw_data/Sensory.csv",sep=";")

# Neurotransmitters values

In [4]:

neurotransmitters = {}
neurotransmitters_connectome = set(np.unique(connectome["Neurotransmitter"]))
neurotransmitters_muscle = set(np.unique(muscles["Neurotransmitter"]))
neurotransmitters_sensor = set(np.unique(sensor["Neurotransmitter"]))
neurotransmitters_full = neurotransmitters_connectome.union(neurotransmitters_muscle)
neurotransmitters_full = neurotransmitters_full.union(neurotransmitters_sensor)
neurotransmitters_list = list(neurotransmitters_full)
for neurotransmitter in neurotransmitters_list:
    neurotransmitters[str(neurotransmitter)] = 1
neurotransmitters["GABA"]=-1
neurotransmitters["GABA_GJ"]=-1
p.dump(neurotransmitters, open("connectome_clean_data/Neurotransmiters_2_coefficient.pickle","wb"))


/Users/peigniersergio/miniconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


# Muscular cells to motor

In [3]:
right_muscles = ["MDR","MVR"]
left_muscles = ["MDL","MVL"]
forward_muscles = ['VA'+str(i) for i in range(1,12)]+['DA'+str(i) for i in range(1,10)]
backward_muscles = ['VB'+str(i) for i in range(1,12)]+['DB'+str(i) for i in range(1,8)]
        
muscular_cells = np.unique(muscles["Muscle"])
muscle_2_motor = {}
for muscular_cell in muscular_cells:
    for rm in right_muscles:
        if rm in muscular_cell:
            muscle_2_motor[muscular_cell] = "MOTOR_RIGHT"
    for lm in left_muscles:
        if lm in muscular_cell:
            muscle_2_motor[muscular_cell] = "MOTOR_LEFT"
    if muscular_cell in forward_muscles:
        muscle_2_motor[muscular_cell] = "MOTOR_FORWARD"
    if muscular_cell in backward_muscles:
        muscle_2_motor[muscular_cell] = "MOTOR_BACKWARD"
p.dump(muscle_2_motor, open("connectome_clean_data/muscle_2_motor.pickle","wb"))

# Clean neuron to muscle

In [6]:
muscles = muscles[["Neuron", "Muscle", "Number of Connections", "Neurotransmitter"]]
muscles.to_csv("connectome_clean_data/Neurons_to_Muscles.csv")

# Clean connectome

In [7]:
connectome = connectome[["Origin", "Target","Number of Connections","Neurotransmitter"]]
connectome.to_csv("connectome_clean_data/Connectome.csv")

# Sensory to sensors

In [22]:
ultra_sounds = "ULTRA_SOUND"
wifi_signal = "WIFI_SIGNAL"
no_signal = "NO_SIGNAL"
sensory_cells_to_sensors = {}
for sensory_cell in np.unique(sensor["Function"]):
    sensory_cell_clean = "|".join("".join(str(sensory_cell).split()).split(","))
    if "gpg-food" in sensory_cell_clean:
        sensory_cells_to_sensors[sensory_cell_clean] = wifi_signal
    elif "gpg-nose" in sensory_cell_clean:
        sensory_cells_to_sensors[sensory_cell_clean] = ultra_sounds
    else:
        sensory_cells_to_sensors[sensory_cell_clean] = no_signal
p.dump(sensory_cells_to_sensors,open("connectome_clean_data/sensory_2_sensors.pickle","wb"))

# Clean sensory

In [23]:
clean_sensory = ["|".join("".join(str(sensory_cell).split()).split(",")) for sensory_cell in sensor["Function"]]
#neurotransmitter_values = np.asarray(map(neurotransmitters_values.get,sensor["Neurotransmitter"]))
sensor["Function"] = clean_sensory
#sensor["Weight"] = neurotransmitter_values * sensor["Weight"]
sensor = sensor[["Function","Neuron","Weight", "Neurotransmitter"]]
sensor.to_csv("connectome_clean_data/Sensory.csv")

# Full Connectome

In [24]:
#connectome
#muscles
full_connectome_dict = {}

for i,sens in sensor.iterrows():
    if sens["Function"] not in full_connectome_dict:
        full_connectome_dict[sens["Function"]] = {sens["Neuron"]:sens["Weight"]*neurotransmitters[str(sens["Neurotransmitter"])]}
    else:
        if sens["Neuron"] not in full_connectome_dict[sens["Function"]]:
            full_connectome_dict[sens["Function"]][sens["Neuron"]] = sens["Weight"]*neurotransmitters[str(sens["Neurotransmitter"])]
        else:
            full_connectome_dict[sens["Function"]][sens["Neuron"]] += sens["Weight"]*neurotransmitters[str(sens["Neurotransmitter"])]
for i,dendrite in connectome.iterrows():
    if dendrite["Origin"] not in full_connectome_dict:
        full_connectome_dict[dendrite["Origin"]] = {dendrite["Target"]:dendrite["Number of Connections"]*neurotransmitters[str(dendrite["Neurotransmitter"])]}
    else:
        if dendrite["Target"] not in full_connectome_dict[dendrite["Origin"]]:
            full_connectome_dict[dendrite["Origin"]][dendrite["Target"]] = dendrite["Number of Connections"]*neurotransmitters[str(dendrite["Neurotransmitter"])]       
        else:
            full_connectome_dict[dendrite["Origin"]][dendrite["Target"]] += dendrite["Number of Connections"]*neurotransmitters[str(dendrite["Neurotransmitter"])] 
for i,dendrite in muscles.iterrows():
    if dendrite["Neuron"] not in full_connectome_dict:
        full_connectome_dict[dendrite["Neuron"]] = {dendrite["Muscle"]:dendrite["Number of Connections"]*neurotransmitters[str(dendrite["Neurotransmitter"])]}
    else:
        if dendrite["Muscle"] not in full_connectome_dict[dendrite["Neuron"]]:
            full_connectome_dict[dendrite["Neuron"]][dendrite["Muscle"]] = dendrite["Number of Connections"]*neurotransmitters[str(dendrite["Neurotransmitter"])]
        else:
            full_connectome_dict[dendrite["Neuron"]][dendrite["Muscle"]] += dendrite["Number of Connections"]*neurotransmitters[str(dendrite["Neurotransmitter"])]
p.dump(full_connectome_dict,open("connectome_clean_data/connectome_dict.pickle","wb"))

In [25]:
full_connectome_dict['VD12']['VA12']

-2

In [4]:
muscle_2_motor

{'MDL01': 'MOTOR_LEFT',
 'MDL02': 'MOTOR_LEFT',
 'MDL03': 'MOTOR_LEFT',
 'MDL04': 'MOTOR_LEFT',
 'MDL05': 'MOTOR_LEFT',
 'MDL06': 'MOTOR_LEFT',
 'MDL07': 'MOTOR_LEFT',
 'MDL08': 'MOTOR_LEFT',
 'MDL09': 'MOTOR_LEFT',
 'MDL10': 'MOTOR_LEFT',
 'MDL11': 'MOTOR_LEFT',
 'MDL12': 'MOTOR_LEFT',
 'MDL13': 'MOTOR_LEFT',
 'MDL14': 'MOTOR_LEFT',
 'MDL15': 'MOTOR_LEFT',
 'MDL16': 'MOTOR_LEFT',
 'MDL17': 'MOTOR_LEFT',
 'MDL18': 'MOTOR_LEFT',
 'MDL19': 'MOTOR_LEFT',
 'MDL20': 'MOTOR_LEFT',
 'MDL21': 'MOTOR_LEFT',
 'MDL22': 'MOTOR_LEFT',
 'MDL23': 'MOTOR_LEFT',
 'MDL24': 'MOTOR_LEFT',
 'MDR01': 'MOTOR_RIGHT',
 'MDR02': 'MOTOR_RIGHT',
 'MDR03': 'MOTOR_RIGHT',
 'MDR04': 'MOTOR_RIGHT',
 'MDR05': 'MOTOR_RIGHT',
 'MDR06': 'MOTOR_RIGHT',
 'MDR07': 'MOTOR_RIGHT',
 'MDR08': 'MOTOR_RIGHT',
 'MDR09': 'MOTOR_RIGHT',
 'MDR10': 'MOTOR_RIGHT',
 'MDR11': 'MOTOR_RIGHT',
 'MDR12': 'MOTOR_RIGHT',
 'MDR13': 'MOTOR_RIGHT',
 'MDR14': 'MOTOR_RIGHT',
 'MDR15': 'MOTOR_RIGHT',
 'MDR16': 'MOTOR_RIGHT',
 'MDR17': 'MOTOR_RIGHT',

In [5]:
muscles

,Neuron,Muscle,Number of Connections,Neurotransmitter,Unnamed: 4
0,ADEL,MDL05,1,Dopamine,NaN
1,AS1,MDL05,3,Acetylcholine,NaN
2,AS1,MDR05,3,Acetylcholine,NaN
3,AS1,MDL08,3,Acetylcholine,NaN
4,AS1,MDR08,4,Acetylcholine,NaN
5,AS10,MDL20,2,Acetylcholine,NaN
6,AS10,MDR20,2,Acetylcholine,NaN
7,AS10,MDL19,3,Acetylcholine,NaN
8,AS10,MDR19,3,Acetylcholine,NaN
9,AS11,MDL21,1,Acetylcholine,NaN
